# 1: Read in the file chop it and make a new one
- Read the csv file
- Create DataFrame df_violin, add in the Day column
- Then for each Day that we are interested in pop into df1, get the subset of data, i.e. WBC0, CRP0, PCT0 for day 0
- Rename the columns in df1 back to the names without day number WBC, CRP, PCT...
- Then append that days data onto the df_violin dataframe with the added column for Day number
- The data is appearing as stirngs instead of numbers, so convert each column to be numeric type or NaN
- finally crate the DatFrame called df , simply because it's shorte variable name than df_violin

In [56]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go

from pathlib import Path
downloads = str(Path.home() / "Downloads")

pd.set_option('display.max_columns', 200)

# read the file
df = pd.read_csv(f"{downloads}/combinedmastersheet1stwavevsGUHadmissions.csv")

# create the df_violin Dataframe
violin_cols = ['Wave', 'Day', 'WBC', 'CRP', 'PCT', 'Fib', 'Platelets', 'Ferritin']
df_violin = pd.DataFrame(columns = violin_cols)

# Get date for each day
for i in [0, 3, 7, 10, 14, 21, 28]:
    cols = ['Wave']
    # set the column names for the relvant day
    for var in ['WBC', 'CRP', 'PCT', 'Fib', 'Platelets', 'Ferritin']:
        cols.append(f'{var}{i}')
    # get temporary df1 using those columns for the current day
    df1 = df[cols]
    # set the Day column value
    df1['Day'] = i
    # now rename all data columns in df1
    for var in ['WBC', 'CRP', 'PCT', 'Fib', 'Platelets', 'Ferritin']:
        df1.rename(columns={ f'{var}{i}' : f'{var}' }, inplace=True)
    # append the df1 onto df_violin
    df_violin = df_violin.append(df1)

# set df to contain df_violin
df = df_violin
# change all fields to be numeric instead of string data
for col in violin_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

<ipython-input-56-5bd96324dc47>:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Create Violin Chart (This is mixed wave 1 and wave 2)

In [83]:
with open('violin_mixed.html', 'w') as f:

    for var in ['WBC', 'CRP', 'PCT', 'Fib', 'Platelets', 'Ferritin']:
        fig = go.Figure()
        for day in [0, 3, 7, 10, 14, 21, 28]:
            fig.add_trace(go.Violin(x=df['Day'][df['Day'] == day],
                                 y=df[var][df['Day'] == day],
                                 box_visible=True,
                                 name = f"Day-{day}",
                                 meanline_visible=True, opacity=0.6))

        fig.update_layout(title_text=f"{var} Combined Wave Violin Chart",
                              xaxis_title="Day",
                              yaxis_title=f"{var} - value",
                          yaxis_zeroline=False)
        fig.show()
        f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))


# Create side by side Violin charts for the Wave 1 and Wave 2

In [84]:
with open('violin_side_by_side.html', 'w') as f:
    for var in ['WBC', 'CRP', 'PCT', 'Fib', 'Platelets', 'Ferritin']:
        fig = go.Figure()

        fig.add_trace(go.Violin(x=df['Day'][ df['Wave'] == 1 ],
                                y=df[var][ df['Wave'] == 1 ],
                                legendgroup='1', scalegroup='1', name='Wave-1',
                                line_color='blue')
                     )
        fig.add_trace(go.Violin(x=df['Day'][ df['Wave'] == 2 ],
                                y=df[var][ df['Wave'] == 2 ],
                                legendgroup='2', scalegroup='2', name='Wave-2',
                                line_color='orange')
                     )

        fig.update_traces(box_visible=True, meanline_visible=True)
        fig.update_layout(title_text=f"{var} Side by Side Violin Chart",
                              xaxis_title="Day",
                              yaxis_title=f"{var} - value",
                          violinmode='group')
        fig.show()
        f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

# Create the split Violin Charts to show Wave1 (left) and Wave2 (right)

In [85]:
with open('violin_split.html', 'w') as f:
    for var in ['WBC', 'CRP', 'PCT', 'Fib', 'Platelets', 'Ferritin']:
        fig = go.Figure()

        fig.add_trace(go.Violin(x=df['Day'][ df['Wave'] == 1 ],
                                y=df[var][ df['Wave'] == 1 ],
                                legendgroup='Wave 1', scalegroup='Wave 1', name='Wave 1',
                                side='negative', box_visible=True,
                                line_color='blue')
                     )
        fig.add_trace(go.Violin(x=df['Day'][ df['Wave'] == 2 ],
                                y=df[var][ df['Wave'] == 2 ],
                                legendgroup='Wave 2', scalegroup='Wave 2', name='Wave 2',
                                side='positive', box_visible=True,
                                line_color='orange')
                     )
        fig.update_traces(meanline_visible=True)
        fig.update_layout(title_text=f"{var} Split Violin Chart",
                              xaxis_title="Day",
                              yaxis_title=f"{var} - value",
                              violingap=0, violinmode='overlay')
        fig.show()
        f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))